In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn import preprocessing
%pylab inline


/Users/davehiltbrand/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Populating the interactive namespace from numpy and matplotlib


In [2]:
#Search directory for all jpgs and returns a list
mylist = os.listdir("./morph/images/")
#Converts list into numpy array
mylist = np.array(mylist)
##Builds a an array of all zeros
lab = np.zeros((len(mylist),), dtype=np.int)
lab = np.resize(lab,(1000,2))

In [3]:
#Searchs strings in mylist for M (male) and returns value of 1 if exists and 0 for female if doesn't exist
for i in range(len(mylist)):
    if "M" in mylist[i]:
        lab[i,0] = 1
        lab[i,1] = 0
    else:
        lab[i,0] = 0
        lab[i,1] = 1


In [4]:
y_train = lab[:,0]
print(y_train.shape)

(1000,)


In [5]:
#Find list of all filenames in directory
import glob
filelist = glob.glob('./morph/images/*.JPG')
#Convert images into numpy arrays
raw = np.array([np.array(Image.open(fname)) for fname in filelist])
#Transforms the 3d array into a 2d array (1000,4200)
raw = raw.reshape((raw.shape[0],-1))
print(raw.shape)

(1000, 4200)


In [6]:
test_size = 0.2
testing_size = int(test_size*len(raw))


train_x = np.array(raw[:,0:4200][:-testing_size]).astype(np.float32)
train_y = np.array(lab[:,0][:-testing_size]).reshape(-1, 1).astype(np.float32)

test_x = np.array(raw[1:,0:4200][-testing_size:]).astype(np.float32)
test_y = np.array(lab[:,0][-testing_size:]).reshape(-1, 1).astype(np.float32)

scaler = preprocessing.MinMaxScaler(feature_range=(-5,5))
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

print(train_y.shape,test_y.shape)

(800, 1) (200, 1)


In [13]:
model_dir='models/morphDNN1'

config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config

#feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_x)

#classifier = tf.estimator.DNNClassifier(
    #feature_columns=feature_cols, hidden_units=[10, 20, 10], n_classes=2, model_dir=model_dir)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=2,    feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)  # if TensorFlow >= 1.1

dnn_clf.fit(train_x, y_train, batch_size=50, steps=100000)


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'models/morphDNN1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c22b9a710>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))

classifier.train(input_fn=dataset, steps=500)

TypeError: unsupported callable

In [9]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(test_x)
accuracy_score(test_y, y_pred['classes'])

INFO:tensorflow:Restoring parameters from /var/folders/fm/z2chdhwn5z3__xvw4bw758hm0000gn/T/tmp6sis6qpk/model.ckpt-100000


0.875

In [10]:
from sklearn.metrics import log_loss

y_pred_proba = y_pred['probabilities']
log_loss(test_y, y_pred_proba)

0.8346482843934636

In [11]:
y_train = np.ndarray.flatten(train_y)
y_train.shape

(800,)